# Label side

In [ ]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-03-01"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

: 

In [2]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np
from jesse import helpers, research

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2715840, 6)

In [2]:
from custom_indicators.toolbox.dollar_bar import estimate_dollar_bar_threshold

for i in [15, 60, 60 * 4]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

15m: 114849881.76817401
60m: 516824464.45678306
240m: 2124722795.2112193


In [3]:
from jesse import helpers

from custom_indicators.config import (
    DOLLAR_BAR_THRESHOLD_LONG,
    DOLLAR_BAR_THRESHOLD_SHORT,
)
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

tp = helpers.date_to_timestamp("2025-01-31")
dollar_bar_15m = build_dollar_bar(
    trading_1m[trading_1m[:, 0] <= tp], DOLLAR_BAR_THRESHOLD_SHORT
)
dollar_bar_4h = build_dollar_bar(
    trading_1m[trading_1m[:, 0] <= tp], DOLLAR_BAR_THRESHOLD_LONG
)
print(dollar_bar_15m.shape, dollar_bar_4h.shape)

np.save("data/dollar_bar_15m.npy", dollar_bar_15m)
np.save("data/dollar_bar_4h.npy", dollar_bar_4h)

(177382, 6) (11225, 6)


In [4]:
from custom_indicators.config import DOLLAR_BAR_THRESHOLD_MID
from custom_indicators.toolbox.dollar_bar import build_dollar_bar

candles_dollar_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_MID)
candles_dollar_bar.shape

(45861, 6)

In [5]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(candles_dollar_bar[:, 2], 0.02)
np.unique(res, return_counts=True)

(array([-1,  1]), array([22107, 23754]))

In [6]:
label = np.column_stack((candles_dollar_bar, res))
label

array([[ 1.57790172e+12,  7.23527000e+03,  7.13415000e+03, ...,
         7.10100000e+03,  7.06863530e+04, -1.00000000e+00],
       [ 1.57797666e+12,  7.13414000e+03,  6.89700000e+03, ...,
         6.86344000e+03,  7.23372420e+04, -1.00000000e+00],
       [ 1.57801764e+12,  6.89700000e+03,  7.26900000e+03, ...,
         6.89631000e+03,  7.07621870e+04,  1.00000000e+00],
       ...,
       [ 1.74077448e+12,  8.38100000e+04,  8.41425000e+04, ...,
         8.35705000e+04,  6.15899600e+03, -1.00000000e+00],
       [ 1.74077640e+12,  8.41400000e+04,  8.43209000e+04, ...,
         8.39547000e+04,  6.14875900e+03, -1.00000000e+00],
       [ 1.74078126e+12,  8.43208000e+04,  8.42996000e+04, ...,
         8.38306000e+04,  6.03447200e+03, -1.00000000e+00]])

In [7]:
np.save("data/label_side_1h.npy", label[label[:, 0] <= tp])
print(label.shape)
print(label[label[:, 0] <= tp].shape)

(45861, 7)
(44867, 7)


In [8]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(
    candles_dollar_bar[candles_dollar_bar[:, 0] <= tp], name_date="time"
)
df_dollar_bar["zigzag"] = res[candles_dollar_bar[:, 0] <= tp]
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-01 18:02:00,2020-01-01 18:02:00,7235.27,7134.15,7242.0,7101.0,70686.353,-1


In [9]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

2025-03-17 00:21:55.861 python[73236:10507674] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-17 00:21:55.861 python[73236:10507674] +[IMKInputSession subclass]: chose IMKInputSession_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop

# Label meta

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

label_side_1h = np.load("data/label_side_1h.npy")
print(label_side_1h.shape)
print(np.unique(label_side_1h[:, 6], return_counts=True))

df_features = pd.read_parquet("data/features_1h.parquet")
print(df_features.shape)

side_model = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob
side_model_pred = np.where(side_model_pred_prob > 0.5, 1, -1)
print(np.unique(side_model_pred, return_counts=True))

(44867, 7)
(array([-1.,  1.]), array([20953, 23914]))
(44867, 11841)
(array([-1,  1]), array([21794, 23073]))


In [2]:
meta_label = (side_model_pred.astype(int) == label_side_1h[:, 6].astype(int)).astype(
    int
)
np.unique(meta_label, return_counts=True)

(array([0, 1]), array([10495, 34372]))

In [3]:
label_meta_1h = label_side_1h.copy()
label_meta_1h[:, 6] = meta_label
np.save("data/label_meta_1h.npy", label_meta_1h)